In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import linregress

# 1.1 Základný opis dát spolu s ich charakteristikami, EDA
## A - Analýza štruktúr dát ako súbory

In [ ]:
# load the datasets
df_profiles     = pd.read_csv("data/raw/profiles.csv", sep='\t')
df_processes    = pd.read_csv("data/raw/processes.csv", sep='\t')
df_devices      = pd.read_csv("data/raw/devices.csv", sep='\t')
df_connections  = pd.read_csv("data/raw/connections.csv", sep='\t')

# better info()
def analyze_dataframe(df):
    table = pd.DataFrame({
        'Missing Values': df.isnull().sum(),
        'Data Type': df.dtypes
    })

    rows, columns = df.shape
    duplicates = df.duplicated().sum()

    print(f"Shape: {rows} rows, {columns} columns")
    print(f"Duplicates: {duplicates}")

    display(table)

## PROFILES:

In [ ]:
# displaying them
analyze_dataframe(df_profiles)
df_profiles.describe()

In [ ]:
df_profiles.head()

## ----------------------------------------------------------------------------------------

## PROCESSES:

In [ ]:
analyze_dataframe(df_processes)
df_processes.describe()

In [ ]:
df_processes.head()

## ----------------------------------------------------------------------------------------

## DEVICES:

In [ ]:
analyze_dataframe(df_devices)
df_devices.describe()

In [ ]:
df_devices.head()

## ----------------------------------------------------------------------------------------

## CONNECTIONS:

In [ ]:
analyze_dataframe(df_connections)
df_connections.describe()

In [ ]:
df_connections.head()

## ----------------------------------------------------------------------------------------

### Zhrnutie analýzy dát
- #### Identifikovali sme duplicitne údajoe a chýbajúce hodnoty. Plánujeme ich odstrániť v nasledujúcej fáze
- #### Budeme sa sústrediť na systémové a používateľské aplikácie, pretože obsahujú relevantné údaje pre detekciu malware

## ----------------------------------------------------------------------------------------
## ----------------------------------------------------------------------------------------

## B - Analýza jednotlivých atribútov, významné atribúty, ich distribúcie a základné deskriptívne štatistiky

#### predovšetkým sa budeme zaujímať o analýzu dataframe-ov: df_processes a df_connections, ktoré súvisia so závislou premennou mwra najviac, vyjadríme si závislosti cez korelačné matice

### DF_PROCESSES

In [ ]:
df_numerical_processes = df_processes.select_dtypes(include=['float64', 'int64'])

correlation_matrix_processes = df_numerical_processes.corr()
correlation_matrix_processes

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(correlation_matrix_processes, annot=True, cmap="coolwarm", fmt='.2f')
plt.title("Processes")
plt.show()

#### z týchto atribútov vyberieme:
- #### p.android.gm
- #### p.android.documentsui
- #### p.system

- #### p.android.externalstorage
- #### p.android.settings
- #### p.android.chrome

#### kôli ich prepojeniu s mwra

In [ ]:
significant_processes = [
    "p.android.gm", 
    "p.android.documentsui",
    "p.system",
    "p.android.externalstorage",
    "p.android.settings",
    "p.android.chrome"]

descriptive_stats = df_processes[significant_processes].describe()
descriptive_stats

#### pomocou histogramov, zisťujeme normalitu týchto atribútov, vieme približne určit aj odľahlé hodnoty, ktoré identifikujeme nižšie

In [ ]:
for attribute in significant_processes:
    plt.figure(figsize=(10, 6))
    sns.histplot(df_processes[attribute], bins=30, kde=True)
    plt.title(f'distribucia: {attribute}')
    plt.xlabel(attribute)
    plt.ylabel('frekvencia')
    plt.show()

#### pomocou boxplotov identifikujeme odľahlé hodnoty a neskôr ich budeme skúmať resp. ich odstránime

In [ ]:
for attribute in significant_processes:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df_processes[attribute])
    plt.title(f'boxplot: {attribute}')
    plt.xlabel(attribute)
    plt.show()

## ----------------------------------------------------------------------------------------

### DF_CONNECTIONS

In [ ]:
df_numerical_connections = df_connections.select_dtypes(include=['float64', 'int64'])

correlation_matrix_connections = df_numerical_connections.corr()
correlation_matrix_connections

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(correlation_matrix_connections, annot=True, cmap="coolwarm", fmt='.2f')
plt.title("Connections")
plt.show()

#### z týchto atribútov vyberáme:
- #### c.katana
- #### c.dogalize
- #### c.adroid.gm
- #### c.android.chrome
- 
#### kôli ich prepojeniu s mwra

In [ ]:
significant_connections = [
    "c.katana",
    "c.dogalize",
    "c.android.chrome",
    "c.android.gm"
]

descriptive_stats = df_connections[significant_connections].describe()
descriptive_stats

#### takisto ako pre df_processes, tak aj tu overujeme normalitu atribútov

In [ ]:
for attribute in significant_connections:
    plt.figure(figsize=(10, 6))
    sns.histplot(df_connections[attribute], bins=30, kde=True)
    plt.title(f'distribucia: {attribute}')
    plt.xlabel(attribute)
    plt.ylabel('frekvencia')
    plt.show()

#### a na identifikáciu odľahlých hodnôt použijeme takisto boxploty

In [ ]:
for attribute in significant_connections:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df_connections[attribute])
    plt.title(f'boxplot: {attribute}')
    plt.xlabel(attribute)
    plt.show()

## ----------------------------------------------------------------------------------------
## ----------------------------------------------------------------------------------------

## C - Párová analýza dát
#### spojíme všetky zvolené významné atribúty do dataframe-u, pre ľahšiu prácu s nimi a následne ich porovnáme, resp. zistíme ich korelácie

In [ ]:
df_merged = pd.merge(df_connections, df_processes, on=['imei', 'ts'], how='inner')
df_merged = df_merged[significant_processes + significant_connections]
df_merged.to_csv('merged_data.csv', index=False)
df_merged.head()

In [ ]:
df_numerical_merged = df_merged.select_dtypes(include=['float64', 'int64'])

correlation_matrix_connections = df_numerical_merged.corr()
correlation_matrix_connections

In [ ]:
sns.heatmap(correlation_matrix_connections, annot=True, cmap="coolwarm", fmt='.2f')

#### porovnávame aj podobné atribúty za predpokladu, že sa hodnoty budú rovnať

In [ ]:
pd.DataFrame({'c.katana': df_connections['c.katana'], 'p.katana': df_processes['p.katana']}).hist(figsize=(6,3))
pd.DataFrame({'c.android.chrome': df_connections['c.android.chrome'], 'p.android.chrome': df_processes['p.android.chrome']}).hist(figsize=(6,3))
pd.DataFrame({'c.dogalize': df_connections['c.dogalize'], 'p.dogalize': df_processes['p.dogalize']}).hist(figsize=(6,3))
pd.DataFrame({'c.android.gm': df_connections['c.android.gm'], 'p.android.gm': df_processes['p.android.gm']}).hist(figsize=(6,3))
pd.DataFrame({'c.android.vending': df_connections['c.android.vending'], 'p.android.vending': df_processes['p.android.vending']}).hist(figsize=(6,3))

#### porovnateľné hodnoty sú, .gm, .chrome, .vending

## D - závislosti medzi predikovanou premennou a ostatnými premennými (sme identifikovali v C) pomocou heatmapy

## Naše prvotné úvahy k riešeniu zadania:
- Predpokladáme, že viaceré atribúty sú medzi sebou prepojené a závislé, napríklad katana a chrome a podobne.
- Závislosť od viacerých atribútov nás viedla k tomu, že sme ich spojili do datasetu df_merged.
- Bude potrebné kombinovať záznamy z viacerých súborov, čo sme už čiastočne vykonali, avšak musíme ešte dôkladnejšie preskúmať závislosti medzi rôznymi atribútmi a následne riešiť ďalšie problémy.

# 1.2 Identifikácia problémov, integrácia a čistenie dát
## A - prvotne riesenia problemov

#### vymažeme duplikáty, keďže niesú vôbec potrebné a môžu výsledky ovplivniť

In [ ]:
df_profiles.info()

In [ ]:
df_profiles['registration']=pd.to_datetime(df_profiles['registration'],format="mixed")
df_profiles['birthdate']=pd.to_datetime(df_profiles['birthdate'])
df_profiles.info()

In [ ]:
df_connections.info()

In [ ]:
df_connections['ts']=pd.to_datetime(df_connections['ts'])
df_connections.info()

In [ ]:
df_processes['ts']=pd.to_datetime(df_processes['ts'])
df_processes.info()

In [ ]:
print(df_connections.duplicated().sum())
print(df_devices.duplicated().sum())
print(df_processes.duplicated().sum())
print(df_profiles.duplicated().sum())

In [ ]:
df_connections = df_connections.drop_duplicates()
df_devices = df_devices.drop_duplicates()
df_processes = df_processes.drop_duplicates()
df_profiles = df_profiles.drop_duplicates()

In [ ]:
print(df_connections.duplicated().sum())
print(df_devices.duplicated().sum())
print(df_processes.duplicated().sum())
print(df_profiles.duplicated().sum())

## B - Missing Values, kde najviac detekovaných missing values sa nachádzalo v df profiles

#### preto sme chýbajúce hodnoty riešili troma spôsobmi:
- v stĺpci birthday sme nahradili chýbajúce hodnoty .mean() - pretože sme to ináč riešiť nevedeli a nechceli sme tento stĺpec vymazať, keďže si myslíme, že vek môže ovplivniť mwra
- v stĺpci job, sme nahradili chýbajúce hodnoty objektom "no-job"
- ostatné nepotrebné stĺpce sme odstránili

In [ ]:
df_profiles.isnull().sum()

In [ ]:
df_profiles['birthdate']=df_profiles['birthdate'].fillna(df_profiles['birthdate'].mean())
df_profiles['birthdate'].isnull().shape[0]
df_profiles.head()

In [ ]:
df_profiles['job'] = df_profiles['job'].fillna('no job')
df_profiles.head()

In [ ]:
for column in df_profiles.columns:
    if df_profiles[df_profiles[column].isnull()].shape[0]:
        df_profiles.drop([column],axis=1,inplace=True)
        print("dropped: "+column)
df_profiles.isnull().sum()

## C - outliers
#### už vyššie sme identifikovali vychýlené hodnoty, v tomto bode sme ich riešili dvoma spôsobmi:
- nahradením
- vymazaním

In [ ]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
androidgm0 = df_connections[df_connections['mwra'] == 0]['c.android.gm']
androidgm0.describe()

In [ ]:
androidgm0_out = identify_outliers(androidgm0)
androidgm0_out

In [ ]:
sns.histplot(androidgm0)

In [ ]:
androidgm0 = androidgm0.drop(androidgm0_out.index)
sns.histplot(androidgm0)

In [ ]:
androidgm1 = df_connections[df_connections['mwra'] == 1]['c.android.gm']
androidgm1.describe()

In [ ]:
sns.histplot(androidgm1)

In [ ]:
androidgm1_out = identify_outliers(androidgm1)
androidgm1_out

In [ ]:
androidgm1 = androidgm1.drop(androidgm1_out.index)
sns.histplot(androidgm1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_merged['c.katana'])
plt.title(f'boxplot: c.katana (povodny)')
plt.xlabel('c.katana')
plt.show()

In [ ]:
z_scores = np.abs(stats.zscore(df_merged['c.katana']))
threshold = 3
df_no_outliers = df_merged['c.katana'][(z_scores < threshold)]

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_no_outliers)
plt.title(f'boxplot: c.katana (bez outlierov)')
plt.xlabel('c.katana')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_merged['c.android.chrome'])
plt.title(f'boxplot: c.android.chrome (povodny)')
plt.xlabel('c.android.chrome')
plt.show()

In [ ]:
lower_bound = df_merged['c.android.chrome'].quantile(0.05)
upper_bound = df_merged['c.android.chrome'].quantile(0.95)

df_replaced_outliers = df_merged['c.android.chrome'].copy()
df_replaced_outliers[df_replaced_outliers < lower_bound] = lower_bound
df_replaced_outliers[df_replaced_outliers > upper_bound] = upper_bound

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_replaced_outliers)
plt.title(f'boxplot: c.android.chrome (outliery nahradené kvantilmi)')
plt.xlabel('c.android.chrome')
plt.show()

# 1.3 Formulácia a štatistické overenie hypotéz o dátach

### Hypoteza 1:
**H₀**: Medzi aplikačnými procesmy bez prítomnosti malvéru (mwra = 0) a s prítomnosťou škodlivého softvéru (mwra = 1) neexistuje signifikantný rozdiel v hodnotách sledovanej premennej 'c.android.gm' (čo zodpovedá aplikácii Gmail pre Android).

In [ ]:
plt.figure(figsize=(10, 6))  
sns.boxplot(x='mwra', y='c.android.gm', data=df_connections)

plt.title('c.android.gm vs mwra')
plt.xlabel('mwra (0 = bez malware, 1 = s malware)')
plt.ylabel('c.android.gm')
plt.grid(axis='y')
plt.show()

In [ ]:
group1 = df_connections[df_connections['mwra'] == 0]['c.android.gm']
group2 = df_connections[df_connections['mwra'] == 1]['c.android.gm']
t_stat, p_value = stats.ttest_ind(group1, group2)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

In [ ]:
from statsmodels.stats.power import TTestIndPower


effect_size = (np.mean(group1) - np.mean(group2)) / np.sqrt((np.var(group1) + np.var(group2)) / 2) 
alpha = 0.05 
n1 = len(group1)  
n2 = len(group2)  

analysis = TTestIndPower()
power = analysis.solve_power(effect_size=effect_size, nobs1=n1, alpha=alpha, ratio=n2/n1, alternative='two-sided')

print(f'Statistická sila testu pre 1. Hypotézu je {power}')


**Záver:** Keďže p-hodnota **(~0.873)** je väčšia ako štandardná hladina významnosti (napr. 0,05), nemáme dostatok dôkazov na zamietnutie nulovej hypotézy.

**Interpretácia:** Z toho vyplýva, že neexistuje štatisticky významný rozdiel medzi zariadeniami s malvérom (mwra = 1) v súvislosti s 'c.android.gm', aplikáciou Gmail.
## ----------------------------------------------------------------------------------------

### Hypotéza 2:
**H₀**: Medzi aplikačnými procesmi bez prítomnosti malvéru (mwra = 0) a s prítomnosťou škodlivého softvéru (mwra = 1) neexistuje signifikantný rozdiel v hodnotách sledovanej premennej 'p.android.settings' (čo zodpovedá nastaveniam Androidu).

In [ ]:
plt.figure(figsize=(10, 6))  
sns.boxplot(x='mwra', y='p.android.settings', data=df_processes)

plt.title('p.android.settings vs mwra')
plt.xlabel('mwra (0 = bez malware, 1 = s malware)')
plt.ylabel('p.android.settings')
plt.grid(axis='y')

plt.show()

group1 = df_processes[df_processes['mwra'] == 0]['p.android.settings']
group2 = df_processes[df_processes['mwra'] == 1]['p.android.settings']
t_stat, p_value = stats.ttest_ind(group1, group2)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

***Záver***: Keďže p-hodnota je menšia ako štandardná hladina významnosti (p < 0,05), máme dostatok dôkazov na zamietnutie nulovej hypotézy.

***Interpretácia***: Z toho vyplýva, že existuje štatisticky významný rozdiel medzi zariadeniami s malvérom (mwra = 1) v súvislosti s aplikáciou 'p.android.settings' (nastavenia Androidu).

In [ ]:
from statsmodels.stats.power import TTestIndPower


effect_size = (np.mean(group1) - np.mean(group2)) / np.sqrt((np.var(group1) + np.var(group2)) / 2)  # efekt veľkosti
alpha = 0.05  
n1 = len(group1)  
n2 = len(group2)  

analysis = TTestIndPower()
power = analysis.solve_power(effect_size=effect_size, nobs1=n1, alpha=alpha, ratio=n2/n1, alternative='two-sided')

print(f'Statistická sila testu pre 2. Hypotézu je {power}')
